In [1]:
from google.colab import drive; drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#pip install --upgrade tensorflow-gpu
#!pip install Keras==2.2.4
!pip install tokenizers

In [3]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
#import sentencepiece as spm
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tokenizers import ByteLevelBPETokenizer
print("Tf",tf.__version__)

Tf 2.3.0


In [4]:
pip install transformers

In [5]:
from transformers import *

In [6]:
  from transformers import XLMRobertaTokenizer
  from transformers import XLMRobertaModel

In [7]:
MAX_LEN = 96  
AUTO = tf.data.experimental.AUTOTUNE
PATH = "/content/drive/My Drive/Gemastik/XLMRoberta"

tokenizer = XLMRobertaTokenizer.from_pretrained("jplu/tf-xlm-roberta-large",add_prefix_space = True,lowercase = True)

In [8]:
 traindf = pd.read_csv("/content/drive/My Drive/Gemastik/PreprocessedManual.csv")
 traindf.drop(["Unnamed: 0"],axis = 1,inplace = True)
 #traindf = pd.read_csv("/content/drive/My Drive/Gemastik/DatasetLabeled.csv",delimiter = "\t")
 #traindf.head()
 #traindf5000 = traindf[:5000]

In [9]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        truncation=True,  
        return_token_type_ids=True,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [10]:
x_train,x_val,y_train,y_val = train_test_split(traindf["full_text"],traindf["is_accident"],
                                               test_size = 0.1, random_state = 0)
x_train = regular_encode(x_train,tokenizer,MAX_LEN)
x_val = regular_encode(x_val,tokenizer,MAX_LEN)
print(x_train.shape,x_val.shape,y_train.shape)

(901, 96) (101, 96) (901,)


In [11]:
#x_train = fast_encode(X_train.astype(str),fast_tokenizer,chunk_size=256,maxlen=96)
#x_valid = fast_encode(X_valid.astype(str),fast_tokenizer,chunk_size=256,maxlen=96)

In [12]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(64)
    .prefetch(AUTO)
)

In [13]:
valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_val, y_val))
    .batch(64)
    .cache()
    .prefetch(AUTO)
)

In [20]:
import keras
def main_model():
  config = XLMRobertaConfig.from_pretrained("jplu/tf-xlm-roberta-large")
  config.num_labels = 2
  config.use_bfloat16 = 'use_fp16'
  config.start_tok = "<s>",
  config.end_tok = "</s>",  
  ENCODER = TFXLMRobertaModel.from_pretrained("jplu/tf-xlm-roberta-large",config = config)
  input_ids = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32)
  attention_mask = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32)

  embedding = ENCODER(input_ids)[0]
  #Disini Boleh Di adjust
  #logits = tf.keras.layers.GlobalAveragePooling1D()(embedding)
  logits = tf.keras.layers.Dropout(0.1)(embedding)
  logits = tf.keras.layers.Flatten()(logits)
  logits = tf.keras.layers.Dense(units = 1024,activation = "tanh")(logits)
  logits = tf.keras.layers.Dropout(0.60)(logits)
  logits = tf.keras.layers.Dense(16,activation="tanh")(logits)
  out = tf.keras.layers.Dense(1, activation="sigmoid", name="final_output")(logits)
  model = tf.keras.Model(inputs=[input_ids], outputs=out)

  loss = tf.keras.losses.BinaryCrossentropy()
  optimizer = tf.keras.optimizers.Adam(lr=2e-5)
  metrics=['accuracy']

  for layer in model.layers[:2]:
    layer.trainable = False

  model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
  return model

model = main_model()
model.summary()

Some weights of the model checkpoint at jplu/tf-xlm-roberta-large were not used when initializing TFXLMRobertaModel: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaModel were initialized from the model checkpoint at jplu/tf-xlm-roberta-large.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 96)]              0         
_________________________________________________________________
tfxlm_roberta_model_4 (TFXLM ((None, 96, 1024), (None, 559890432 
_________________________________________________________________
dropout_375 (Dropout)        (None, 96, 1024)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 98304)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              100664320 
_________________________________________________________________
dropout_376 (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)               

In [ ]:
import keras

In [ ]:
'''def build_model(transformer, max_len=512):
    """
    """
    input_word_ids = keras.layers.Input((MAX_LEN,), dtype='int32', name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_out = keras.layers.Dropout(0.5)(sequence_output)
    logits = keras.layers.Dense(units=768, activation="tanh")(cls_out)
    logits = keras.layers.Dropout(0.5)(logits)
    logits = keras.layers.Dense(units=1, activation="sigmoid")(logits)
    model = keras.Model(inputs=input_word_ids, outputs=logits)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model
    '''

In [ ]:
'''def build_model(transformer, max_len=512):
  
    input_word_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model'''

In [ ]:
'''strategy = tf.distribute.get_strategy()
with strategy.scope():
        model = build_model(TFXLMRobertaModel.from_pretrained("jplu/tf-xlm-roberta-base",num_labels = 2),MAX_LEN)'''

In [ ]:
#config=XLMRobertaConfig.from_pretrained("jplu/tf-xlm-roberta-base")
#model = build_model(TFXLMRobertaModel.from_pretrained("jplu/tf-xlm-roberta-base",config= config), max_len=MAX_LEN)
#model.summary()

In [18]:
def create_learning_rate_scheduler(max_learn_rate=5e-5,
                                   end_learn_rate=1e-7,
                                   warmup_epoch_count=10,
                                   total_epoch_count=90):

    def lr_scheduler(epoch):
        if epoch < warmup_epoch_count:
            res = (max_learn_rate/warmup_epoch_count) * (epoch + 1)
        else:
            res = max_learn_rate*math.exp(math.log(end_learn_rate/max_learn_rate)*(epoch-warmup_epoch_count+1)/(total_epoch_count-warmup_epoch_count+1))
        return float(res)
    learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)

    return learning_rate_scheduler

In [ ]:
total_epoch_count = 10
n_steps = x_train.shape[0]
model.fit(x = train_dataset,
          shuffle=True,
          steps_per_epoch=n_steps,
          validation_data=valid_dataset,
          epochs=total_epoch_count,
          callbacks=[create_learning_rate_scheduler(max_learn_rate=1e-5,
                                                    end_learn_rate=1e-7,
                                                    warmup_epoch_count=20,
                                                    total_epoch_count=total_epoch_count),
                     keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
                     ]) 


Epoch 00001: LearningRateScheduler reducing learning rate to 5.000000000000001e-07.
Epoch 1/10
636/901 [====================>.........] - ETA: 6:01 - loss: 0.6754 - accuracy: 0.6188

In [ ]:
#model.save_weights(model_save_location, overwrite=True)